# 🔥 Orman Yangını Tespit Sistemi
## CNN ile Görüntü Sınıflandırma Projesi

Bu notebook, orman yangınlarını tespit etmek için bir CNN modeli eğitir.

## 1. Kütüphaneleri İçe Aktar

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# GPU kontrolü
print("TensorFlow Versiyonu:", tf.__version__)
print("GPU Mevcut mu?:", len(tf.config.list_physical_devices('GPU')) > 0)

## 2. Veri Setini Kontrol Et

In [ ]:
# Veri yolları
train_path = 'train'
test_path = 'test'

# Klasörlerde kaçar tane resim var
def sayi_bul(path):
    if not os.path.exists(path):
        return "Klasör bulunamadı!"
    for sinif in os.listdir(path):
        sinif_yolu = os.path.join(path, sinif)
        if os.path.isdir(sinif_yolu):
            print(f"{path} -> {sinif} sınıfında: {len(os.listdir(sinif_yolu))} resim var.")

print("\n--- Veri Seti Kontrolü ---")
sayi_bul(train_path)
sayi_bul(test_path)

## 3. Veri Hazırlama (ImageDataGenerator)

In [ ]:
# Görüntü ayarları
IMG_SHAPE = 224
BATCH_SIZE = 32

print("Veriler yükleniyor...")

# Resimleri 0-1 arasına sıkıştırma (Rescale)
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Train verisini oku
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_SHAPE, IMG_SHAPE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

# Test verisini oku
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(IMG_SHAPE, IMG_SHAPE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

## 4. CNN Modelini Oluştur

In [ ]:
print("\nModel oluşturuluyor...")

model = Sequential()

# 1. Katman
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SHAPE, IMG_SHAPE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2. Katman
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 3. Katman
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Sınıflandırma
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Yangın Var/Yok (0 veya 1)

# Derleme
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Model özetini göster
model.summary()

## 5. Modeli Eğit

In [ ]:
print("\nEğitim Başlıyor... (Lütfen bekleyiniz)")

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

print("Eğitim Tamamlandı!")

## 6. Eğitim Grafiklerini Çiz

In [ ]:
# Grafik boyutlarını ayarlayalım
plt.figure(figsize=(12, 4))

# --- 1. GRAFİK: BAŞARI (ACCURACY) ---
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Eğitim Başarısı (Train Acc)')
plt.plot(history.history['val_accuracy'], label='Test Başarısı (Validation Acc)')
plt.title('Model Başarı Grafiği')
plt.xlabel('Epoch (Tur Sayısı)')
plt.ylabel('Başarı (Accuracy)')
plt.legend()
plt.grid(True)

# --- 2. GRAFİK: HATA (LOSS) ---
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Eğitim Hatası (Train Loss)')
plt.plot(history.history['val_loss'], label='Test Hatası (Validation Loss)')
plt.title('Model Hata Grafiği')
plt.xlabel('Epoch (Tur Sayısı)')
plt.ylabel('Hata (Loss)')
plt.legend()
plt.grid(True)

# Grafiği göster ve kaydet
plt.savefig('training_history.png')
plt.show()

print("📊 Grafik kaydedildi: training_history.png")

## 7. Sonuçları Değerlendir

In [ ]:
# Son epoch sonuçları
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

print("\n" + "="*50)
print("📊 SONUÇLAR")
print("="*50)
print(f"Train Accuracy: {final_train_acc*100:.2f}%")
print(f"Validation Accuracy: {final_val_acc*100:.2f}%")
print(f"Train Loss: {final_train_loss:.4f}")
print(f"Validation Loss: {final_val_loss:.4f}")
print("="*50)

## 8. Modeli Kaydet

In [ ]:
# Modeli kaydet
model.save('fire_detection_model.h5')
print("💾 Model kaydedildi: fire_detection_model.h5")